# Data Download and Exploration Notebook
## Team 12 Final Project

Step 0: Importing Packages

In [1]:

import pandas as pd
# !pip install wrds
import wrds
import os
import zipfile

c:\Users\ziggy\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


This is a link explaining [the wrds package](https://pypi.org/project/wrds/)

In [2]:
# db = wrds.Connection()
# db.list_libraries()

Donwloading CSVs

In [3]:
ipo_age_df = pd.read_csv('inputs/IPO-age(9).csv')
SPACs_df = pd.read_csv('inputs/SPACs2016-2021.csv')
# ipo_age_df
# SPACs_df

Data Exploration

In [4]:
ipo_age_df.describe()
ipo_age_df.info()
ipo_age_df.nunique()
# unnamed columns = notes in the .csv file, we can drop these
ipo_age_df.drop(columns=['Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13'], inplace=True)
ipo_age_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15447 entries, 0 to 15446
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   offer date         15447 non-null  int64  
 1   IPO name           15447 non-null  object 
 2   ticker             15035 non-null  object 
 3   CUSIP              15317 non-null  object 
 4   Rollup             15447 non-null  object 
 5   VC                 15447 non-null  object 
 6   Dual               15447 non-null  object 
 7   Internet           15447 non-null  object 
 8   Post-issue shares  15447 non-null  object 
 9   CRSP perm          15447 non-null  object 
 10  Founding           15447 non-null  int64  
 11  Unnamed: 11        0 non-null      float64
 12  Unnamed: 12        17 non-null     object 
 13  Unnamed: 13        3 non-null      object 
dtypes: float64(1), int64(2), object(11)
memory usage: 1.7+ MB


,offer date,IPO name,ticker,CUSIP,Rollup,VC,Dual,Internet,Post-issue shares,CRSP perm,Founding
0,19750130,ROYSTER,NaN,78088610,.,0,0,0,.,67898,1901
1,19750609,VARCO,VRC,92212610,.,0,0,0,.,63044,1908
2,19750610,COORS ADOLPH,TAP,21701610,.,0,1,0,35418915,59248,1901
3,19750715,KEYSTONE FOODS CORP,NaN,49348410,.,0,0,0,.,61989,1965
4,19750826,C. F. BRAUN,NaN,10564710,.,0,0,0,.,58579,1909
...,...,...,...,...,...,...,...,...,...,...,...
15442,20231214,INNO HOLDINGS INC.,INHD,4576JP109,.,0,0,0,.,.,2021
15443,20231215,Bayview Acquisition,BAYAU,NaN,.,0,1,0,7732500,.,2023
15444,20231215,ZKH GROUP LTD,ZKH,98877R104,.,0,1,0,5621490964,.,2021
15445,20231219,Linkage Global Inc,LGCB,G5500B102,.,0,0,0,.,.,2022


In [5]:
SPACs_df.describe()
SPACs_df.info()
SPACs_df.nunique() # 6 repeating CUSIPS? 881 total values, 875 unique?

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1014 entries, 0 to 1013
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   SPAC IPO     1013 non-null   object
 1   Date of IPO  1013 non-null   object
 2   CUSIP        881 non-null    object
dtypes: object(3)
memory usage: 23.9+ KB


SPAC IPO       1013
Date of IPO     423
CUSIP           875
dtype: int64

Downloading the ccm_youngfirms data (courtesy of Prof. Bowen)

In [6]:
ccm_youngfirms = "inputs/ccm_youngfirms_2000_2018.dta/ccm_youngfirms_2000_2018.dta"

if not os.path.exists(ccm_youngfirms):
    zip_path = "inputs/ccm_youngfirms_2000_2018.zip"

    with zipfile.ZipFile(zip_path,'r') as zip_ref:
        zip_ref.extractall("inputs/ccm_youngfirms_2000_2018.dta")
        

In [7]:
data = pd.read_stata(ccm_youngfirms)

ccm_youngfirms is huge and we need to be dealing with less than 950 vars

In [8]:
data.dropna(axis=1,how='all',inplace=True)
# 950 to 589 vars --> still too many

.dropna() with these params drops all the cols with 0 non-NA values.

In [24]:
sample = data.iloc[:,0:20]
# sample.nunique()
isna = sample.isna().sum()/len(sample)
isna

lpermno     0.000000
lpermco     0.000000
gvkey       0.000000
datadate    0.000000
fyear       0.000000
indfmt      0.000000
consol      0.000000
popsrc      0.000000
datafmt     0.000000
tic         0.000000
cusip       0.000000
conm        0.000000
acctchg     0.000000
acctstd     0.000000
acqmeth     0.000000
adrr        0.915863
ajex        0.000000
ajp         0.000000
bspr        0.000000
compst      0.000000
dtype: float64

here we drop all the columns with higher than 90% NA values.

In [35]:
na_percentage = data.isna().mean() * 100

# Filter columns where the percentage of NA values is greater than 90%
columns_with_high_na = na_percentage[na_percentage > 90].index.tolist()

print("Columns with more than 90% NA values:")
print(len(columns_with_high_na))

data.drop(columns=columns_with_high_na, inplace=True)
data

Columns with more than 90% NA values:
165


,lpermno,lpermco,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,...,prcc_f,prch_f,prcl_f,adjex_f,rank,au,auop,auopic,ceoso,cfoso
0,85073.0,32010.0,002731,2000-12-31,2000,INDL,C,D,STD,PVN,...,57.50,67.0000,29.0625,1.0,NaN,,,,,
1,85073.0,32010.0,002731,2001-12-31,2001,INDL,C,D,STD,PVN,...,3.55,64.0625,2.0000,1.0,NaN,,,,,
2,85073.0,32010.0,002731,2002-12-31,2002,INDL,C,D,STD,PVN,...,6.49,8.4900,2.5000,1.0,NaN,,,,,
3,85073.0,32010.0,002731,2003-12-31,2003,INDL,C,D,STD,PVN,...,11.64,13.1400,4.8700,1.0,1.0,,,,Y,Y
4,14055.0,54485.0,003169,2013-12-31,2013,INDL,C,D,STD,OMED.2,...,29.52,32.3890,12.0650,1.0,1.0,4,1,0,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27783,17808.0,56369.0,324684,2018-12-31,2018,INDL,C,D,STD,ASLN,...,3.60,10.4390,2.5100,0.2,1.0,5,1,0,Y,Y
27784,17676.0,56329.0,326688,2018-12-31,2018,INDL,C,D,STD,NVT,...,22.46,29.8400,19.7100,1.0,1.0,5,4,1,Y,Y
27785,17884.0,56407.0,327451,2018-12-31,2018,INDL,C,D,STD,GRIN,...,6.24,20.5000,5.3200,1.0,1.0,5,1,0,Y,Y
27786,18313.0,56612.0,328032,2018-12-31,2018,INDL,C,D,STD,NMCI,...,2.83,4.8500,2.5100,1.0,1.0,4,1,0,Y,Y


Merging Data

,lpermno,lpermco,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,...,prcc_f,prch_f,prcl_f,adjex_f,rank,au,auop,auopic,ceoso,cfoso
0,85073.0,32010.0,002731,2000-12-31,2000,INDL,C,D,STD,PVN,...,57.50,67.0000,29.0625,1.0,NaN,,,,,
1,85073.0,32010.0,002731,2001-12-31,2001,INDL,C,D,STD,PVN,...,3.55,64.0625,2.0000,1.0,NaN,,,,,
2,85073.0,32010.0,002731,2002-12-31,2002,INDL,C,D,STD,PVN,...,6.49,8.4900,2.5000,1.0,NaN,,,,,
3,85073.0,32010.0,002731,2003-12-31,2003,INDL,C,D,STD,PVN,...,11.64,13.1400,4.8700,1.0,1.0,,,,Y,Y
4,14055.0,54485.0,003169,2013-12-31,2013,INDL,C,D,STD,OMED.2,...,29.52,32.3890,12.0650,1.0,1.0,4,1,0,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27783,17808.0,56369.0,324684,2018-12-31,2018,INDL,C,D,STD,ASLN,...,3.60,10.4390,2.5100,0.2,1.0,5,1,0,Y,Y
27784,17676.0,56329.0,326688,2018-12-31,2018,INDL,C,D,STD,NVT,...,22.46,29.8400,19.7100,1.0,1.0,5,4,1,Y,Y
27785,17884.0,56407.0,327451,2018-12-31,2018,INDL,C,D,STD,GRIN,...,6.24,20.5000,5.3200,1.0,1.0,5,1,0,Y,Y
27786,18313.0,56612.0,328032,2018-12-31,2018,INDL,C,D,STD,NMCI,...,2.83,4.8500,2.5100,1.0,1.0,4,1,0,Y,Y
